# Comparison CDM with and without numba

In [1]:
import numpy as np
import numba
from pathlib import Path

## Prepare input data

In [3]:
# Input filenames
parallel_filename = Path('../cdm_euclid_parallel.dat').resolve(strict=True)
serial_filename = Path('../cdm_euclid_serial.dat').resolve(strict=True)

# Input frame
shape = 2000, 2000
frame = np.arange(shape[0] * shape[1]).reshape(shape)
frame.dtype

dtype('int64')

In [3]:
# Initialization
parallel_trapdata = np.loadtxt(parallel_filename)
assert parallel_trapdata.ndim > 1

serial_trapdata = np.loadtxt(serial_filename)
assert serial_trapdata.ndim > 1

## Run CDM without 'Numba'

In [4]:
def run_cdm(image, parallel_trapdata, serial_trapdata,             
            rdose: float = 8.0e11,
            vth: float = 1.168e7,
            beta_p: float = 0.6, beta_s: float = 0.6,
            vg: float = 6.e-11, svg: float = 1.0e-10,
            t: float = 20.48e-3, st: float = 5.0e-6,
            fwc: int = 200000, sfwc: int = 730000):
    """Electron trapping in imaging mode (non-TDI).

    :param image:
    :return:
    """
    nt_p = parallel_trapdata[:, 0]
    sigma_p = parallel_trapdata[:, 1]
    tr_p = parallel_trapdata[:, 2]
    
    nt_s = serial_trapdata[:, 0]
    sigma_s = serial_trapdata[:, 1]
    tr_s = serial_trapdata[:, 2]
    
    # absolute trap density which should be scaled according to radiation dose
    # (nt=1.5e10 gives approx fit to GH data for a dose of 8e9 10MeV equiv. protons)

    # array sizes
    ydim, xdim = image.shape
    s = image

    # add background electrons (diffuse optical background level)
    # s += self.dob

    # apply FWC (anti-blooming) - not needed we apply this model elsewhere
    # msk = s > self.fwc
    # s[msk] = self.fwc

    
    # start with parallel direction
#     if True:
        # print('adding parallel')
    no = np.zeros_like(image, dtype=np.float64)
    nt_p *= rdose             # absolute trap density [per cm**3]
    zdim_p = len(nt_p)

    alpha_p = t * sigma_p * vth * fwc ** beta_p / (2. * vg)
    g_p = 2. * nt_p * vg / fwc ** beta_p

    gamma_p = g_p * np.arange(ydim).reshape((ydim, 1))

    for i in range(ydim):
        # print(i)
        for k in range(zdim_p):
            for j in range(xdim):
                nc = 0.

                if s[i, j] > 0.01:
                    nc = max((gamma_p[i, k] * s[i, j] ** beta_p - no[j, k]) /
                             (gamma_p[i, k] * s[i, j] ** (beta_p - 1.) + 1.) *
                             (1. - np.exp(-alpha_p[k] * s[i, j] ** (1. - beta_p))), 0.)

                no[j, k] += nc
                nr = no[j, k] * (1. - np.exp(-t/tr_p[k]))
                s[i, j] += -1 * nc + nr
                no[j, k] -= nr

    # now serial direction
    #if self.serial_cti:
        # print('adding serial')

    sno = np.zeros_like(image, dtype=np.float64)
    nt_s *= rdose             # absolute trap density [per cm**3]
    zdim_s = len(nt_s)

    alpha_s = st * sigma_s * vth * sfwc ** beta_s / (2. * svg)
    g_s = 2. * nt_s * svg / sfwc ** beta_s

    gamma_s = g_s * np.arange(xdim).reshape((xdim, 1))

    for j in range(xdim):
        # print(j)
        for k in range(zdim_s):
            if tr_s[k] < t:
                for i in range(ydim):
                    nc = 0.

                    if s[i, j] > 0.01:
                        nc = max((gamma_s[j, k] * s[i, j] ** beta_s - sno[i, k]) /
                                 (gamma_s[j, k] * s[i, j] ** (beta_s - 1.) + 1.) *
                                 (1. - np.exp(-alpha_s[k] * s[i, j] ** (1. - beta_s))), 0.)

                    sno[i, k] += nc
                    nr = sno[i, k] * (1. - np.exp(-st/tr_s[k]))
                    s[i, j] += -1 * nc + nr
                    sno[i, k] -= nr

    return s

In [5]:
%timeit run_cdm(image=frame.astype(np.float), parallel_trapdata=parallel_trapdata.copy(), serial_trapdata=serial_trapdata.copy())

4min 31s ± 749 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
result1 = run_cdm(image=frame.astype(np.float), parallel_trapdata=parallel_trapdata.copy(), serial_trapdata=serial_trapdata.copy())
result1

array([[0.00000000e+00, 8.73795487e-01, 1.75469425e+00, ...,
        1.98366054e+03, 1.98465760e+03, 1.98565467e+03],
       [1.92038057e+03, 1.90694591e+03, 1.90812763e+03, ...,
        3.85898827e+03, 3.85996777e+03, 3.86094727e+03],
       [3.83837283e+03, 3.81745832e+03, 3.81869169e+03, ...,
        5.78889622e+03, 5.78988173e+03, 5.79086724e+03],
       ...,
       [3.98157507e+06, 3.98014992e+06, 3.98015202e+06, ...,
        3.98249390e+06, 3.98249492e+06, 3.98249595e+06],
       [3.98357135e+06, 3.98214577e+06, 3.98214787e+06, ...,
        3.98448985e+06, 3.98449087e+06, 3.98449190e+06],
       [3.98556763e+06, 3.98414163e+06, 3.98414373e+06, ...,
        3.98648579e+06, 3.98648682e+06, 3.98648785e+06]])

## With Numba

In [7]:
@numba.jit(nopython=True, nogil=True)
def run_cdm_with_numba(image, parallel_trapdata, serial_trapdata,             
            rdose: float = 8.0e11,
            vth: float = 1.168e7,
            beta_p: float = 0.6, beta_s: float = 0.6,
            vg: float = 6.e-11, svg: float = 1.0e-10,
            t: float = 20.48e-3, st: float = 5.0e-6,
            fwc: int = 200000, sfwc: int = 730000):
    """Electron trapping in imaging mode (non-TDI).

    :param image:
    :return:
    """
    nt_p = parallel_trapdata[:, 0]
    sigma_p = parallel_trapdata[:, 1]
    tr_p = parallel_trapdata[:, 2]
    
    nt_s = serial_trapdata[:, 0]
    sigma_s = serial_trapdata[:, 1]
    tr_s = serial_trapdata[:, 2]
    
    # absolute trap density which should be scaled according to radiation dose
    # (nt=1.5e10 gives approx fit to GH data for a dose of 8e9 10MeV equiv. protons)

    # array sizes
    ydim, xdim = image.shape
    s = image

    # add background electrons (diffuse optical background level)
    # s += self.dob

    # apply FWC (anti-blooming) - not needed we apply this model elsewhere
    # msk = s > self.fwc
    # s[msk] = self.fwc

    
    # start with parallel direction
#     if True:
        # print('adding parallel')
    no = np.zeros_like(image, dtype=np.float64)
    nt_p *= rdose             # absolute trap density [per cm**3]
    zdim_p = len(nt_p)

    alpha_p = t * sigma_p * vth * fwc ** beta_p / (2. * vg)
    g_p = 2. * nt_p * vg / fwc ** beta_p

    gamma_p = g_p * np.arange(ydim).reshape((ydim, 1))

    for i in range(ydim):
        # print(i)
        for k in range(zdim_p):
            for j in range(xdim):
                nc = 0.

                if s[i, j] > 0.01:
                    nc = max((gamma_p[i, k] * s[i, j] ** beta_p - no[j, k]) /
                             (gamma_p[i, k] * s[i, j] ** (beta_p - 1.) + 1.) *
                             (1. - np.exp(-alpha_p[k] * s[i, j] ** (1. - beta_p))), 0.)

                no[j, k] += nc
                nr = no[j, k] * (1. - np.exp(-t/tr_p[k]))
                s[i, j] += -1 * nc + nr
                no[j, k] -= nr

    # now serial direction
    #if self.serial_cti:
        # print('adding serial')

    sno = np.zeros_like(image, dtype=np.float64)
    nt_s *= rdose             # absolute trap density [per cm**3]
    zdim_s = len(nt_s)

    alpha_s = st * sigma_s * vth * sfwc ** beta_s / (2. * svg)
    g_s = 2. * nt_s * svg / sfwc ** beta_s

    gamma_s = g_s * np.arange(xdim).reshape((xdim, 1))

    for j in range(xdim):
        # print(j)
        for k in range(zdim_s):
            if tr_s[k] < t:
                for i in range(ydim):
                    nc = 0.

                    if s[i, j] > 0.01:
                        nc = max((gamma_s[j, k] * s[i, j] ** beta_s - sno[i, k]) /
                                 (gamma_s[j, k] * s[i, j] ** (beta_s - 1.) + 1.) *
                                 (1. - np.exp(-alpha_s[k] * s[i, j] ** (1. - beta_s))), 0.)

                    sno[i, k] += nc
                    nr = sno[i, k] * (1. - np.exp(-st/tr_s[k]))
                    s[i, j] += -1 * nc + nr
                    sno[i, k] -= nr

    return s

In [8]:
parallel_trapdata

array([[4.00e+01, 2.20e-13, 8.20e-07],
       [1.20e+00, 2.20e-13, 3.00e-04],
       [1.00e+00, 4.72e-15, 2.00e-03],
       [1.00e+00, 1.37e-16, 2.50e-02],
       [2.30e-01, 2.78e-17, 1.24e-01],
       [2.90e+00, 1.93e-17, 1.67e+01],
       [1.00e+01, 6.39e-18, 4.96e+02]])

In [9]:
%timeit run_cdm_with_numba(image=frame.astype(np.float), parallel_trapdata=parallel_trapdata.copy(), serial_trapdata=serial_trapdata.copy())

6.33 s ± 387 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
result2 = run_cdm_with_numba(image=frame.astype(np.float), parallel_trapdata=parallel_trapdata.copy(), serial_trapdata=serial_trapdata.copy())
result2

array([[0.00000000e+00, 8.73795487e-01, 1.75469425e+00, ...,
        1.98366054e+03, 1.98465760e+03, 1.98565467e+03],
       [1.92038057e+03, 1.90694591e+03, 1.90812763e+03, ...,
        3.85898827e+03, 3.85996777e+03, 3.86094727e+03],
       [3.83837283e+03, 3.81745832e+03, 3.81869169e+03, ...,
        5.78889622e+03, 5.78988173e+03, 5.79086724e+03],
       ...,
       [3.98157507e+06, 3.98014992e+06, 3.98015202e+06, ...,
        3.98249390e+06, 3.98249492e+06, 3.98249595e+06],
       [3.98357135e+06, 3.98214577e+06, 3.98214787e+06, ...,
        3.98448985e+06, 3.98449087e+06, 3.98449190e+06],
       [3.98556763e+06, 3.98414163e+06, 3.98414373e+06, ...,
        3.98648579e+06, 3.98648682e+06, 3.98648785e+06]])

In [11]:
np.testing.assert_array_equal(result1, result2)

## Result comparison

| Frame shape | Without Numba | With Numba | Speedup |
|-------------|---------------|------------|---------|
| 50x50       | 180 ms        | 3.67 ms    | 49x     |
| 100x100     | 687 ms        | 14.3 ms    | 48x     |
| 500x500     | 17.2 s        | 355 ms     | 48x     |
| 1000x1000   | 1 min 15 s    | 1.5 s      | 50x     |
| 2000x2000   | 4 min 31 s    | 6.33 s     | 42x     |